# Import Libraries

In [53]:
import torch, os
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
from torchvision.utils import make_grid
from torch.autograd import Variable
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

import sys
from os import listdir
from os.path import isfile, join
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
import librosa
import librosa.display
import IPython.display as ipd

import random

%matplotlib inline

print(sys.version)
print('pytorch : ', torch.__version__)
print('librosa : ', librosa.__version__)
print('numpy : ', np.__version__)
print('scipy : ', sp.__version__)
print('matplotlib : ', mpl.__version__)

3.6.1 |Anaconda custom (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
pytorch :  0.4.0
librosa :  0.5.1
numpy :  1.12.1
scipy :  1.0.1
matplotlib :  2.0.2


# Hparams Setting

In [10]:
lr = 0.0001
max_epoch = 20
batch_size = 64
z_dim = 100
image_size = 64
g_conv_dim = 64
d_conv_dim = 64
log_step = 100
sample_step = 500
sample_num = 32

# Generator

In [11]:
# Conv2d (Batch_num , Channel, length) 
def deconv(c_in, c_out, k_size, stride=2, pad=1, bn=True):
    """Custom deconvolutional layer for simplicity."""
    layers = []
    layers.append(nn.ConvTranspose2d(c_in, c_out, k_size, stride, pad))
    if bn:
        layers.append(nn.BatchNorm2d(c_out))
    return nn.Sequential(*layers)

#deconv1d
def deconv1d(c_in, c_out, k_size, stride=4, pad=11, out_pad = 1,bn=True):
    """Custom convolutional 1d lyaer for simplicity."""
    layers = []
    layers.append(nn.ConvTranspose1d(c_in, c_out, k_size, stride, pad,out_pad))
    if bn:
        layers.append(nn.BatchNorm1d(c_out))
    return nn.Sequential(*layers)

class WaveGAN_Generator(nn.Module):
    """Generator containing 7 deconvolutional layers."""
    def __init__(self, z_dim=100, image_size=128, conv_dim=g_conv_dim):
        super(WaveGAN_Generator, self).__init__()
        self.fc = nn.Linear(z_dim, 256*conv_dim)
#        self.deconv1 = deconv1d(conv_dim*16, conv_dim*8, (25,2,2), pad = 11)
        self.deconv1 = deconv1d(conv_dim*16, conv_dim*8, k_size = 25, pad = 11, out_pad = 1)
        self.deconv2 = deconv1d(conv_dim*8, conv_dim*4, 25)
        self.deconv3 = deconv1d(conv_dim*4, conv_dim*2, 25)
        self.deconv4 = deconv1d(conv_dim*2, conv_dim, 25)
        self.deconv5 = deconv1d(conv_dim, 1, 25, bn=False)
        
    def forward(self, z):
#        z = z.view(z.size(0), z.size(1))      # If image_size is 64, output shape is as below.
        out = self.fc(z)                 # (?, 256d)
        print(out.size())
        out = out.view(out.size(0),16*g_conv_dim,16 ) # (?,16,16d)
        print(out.size())
        out = F.relu(out)
        out = F.relu(self.deconv1(out))  # (?, 64, 8d)
        print("a")
        print(out.size())
        
        out = F.relu(self.deconv2(out))  # (?, 256, 4d)
        
        print(out.size())
        out = F.relu(self.deconv3(out))  # (?, 1024, 2d)
        print(out.size())
        out = F.relu(self.deconv4(out))  # (?, 4096, d)
        print(out.size())
        out = F.tanh(self.deconv5(out))  # (?, 16384, c)
        print(out.size())
        return out
    
G = WaveGAN_Generator(z_dim,image_size,g_conv_dim)
G

WaveGAN_Generator(
  (fc): Linear(in_features=100, out_features=16384, bias=True)
  (deconv1): Sequential(
    (0): ConvTranspose1d(1024, 512, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv2): Sequential(
    (0): ConvTranspose1d(512, 256, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv3): Sequential(
    (0): ConvTranspose1d(256, 128, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv4): Sequential(
    (0): ConvTranspose1d(128, 64, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv5): Sequential(
    (0

In [12]:
n_data = torch.ones(5,100)
#n_data

In [13]:
wavegan_g = WaveGAN_Generator()

output = wavegan_g.forward(n_data)
output.size()

torch.Size([5, 16384])
torch.Size([5, 1024, 16])
a
torch.Size([5, 512, 64])
torch.Size([5, 256, 256])
torch.Size([5, 128, 1024])
torch.Size([5, 64, 4096])
torch.Size([5, 1, 16384])


torch.Size([5, 1, 16384])

# Discriminator

In [81]:
def conv(c_in, c_out, k_size, stride=4, pad=1,bn=True):
    """Custom convolutional 1d lyaer for simplicity."""
    layers = []
    layers.append(nn.Conv2d(c_in, c_out, k_size,stride,pad))
    if bn:
        layers.append(nn.BatchNorm2d(c_out))
    return nn.Sequential(*layers)


def conv1d(c_in, c_out, k_size, stride=4, pad=11, out_pad = 1,bn=True):
    """Custom convolutional 1d lyaer for simplicity."""
    layers = []
    layers.append(nn.Conv1d(c_in, c_out, k_size,stride,pad))
    if bn:
        layers.append(nn.BatchNorm1d(c_out))
    return nn.Sequential(*layers)

def apply_phaseshuffle(x, n_phase, pad_type = 'refelct'):
    
    #asdf
    (batch, n_channel, x_len) = x.shape
    r = random.randrange(-n_phase, n_phase+1)

    pad_l = np.max(r, 0)
    pad_r = np.max(-r, 0)
    phase_start = pad_r
    
    padding = nn.ReflectionPad2d((pad_l, pad_r, 0, 0))
    
    
    print(pad_l, pad_r, phase_start, x_len)
    print("phase")
    print(x.shape)
    
    for x_ in x:
        print(x_.shape)
        x_ = padding(x_)
        x_ = x[:, phase_start:phase_start + x_len]
    
    print(x.shape)
    
    return x
    
def apply_phaseshuffle2(x, rad, pad_type='reflect'):
    b, x_len, nch = x.get_shape().as_list()

    phase = tf.random_uniform([], minval=-rad, maxval=rad + 1, dtype=tf.int32)
    pad_l = tf.maximum(phase, 0)
    pad_r = tf.maximum(-phase, 0)
    phase_start = pad_r
    x = tf.pad(x, [[0, 0], [pad_l, pad_r], [0, 0]], mode=pad_type)

    x = x[:, phase_start:phase_start+x_len]
    x.set_shape([b, x_len, nch])

    return x    
    

#Ref DCGAN : https://github.com/InsuJeon/Hello-Generative-Model/blob/master/Day04/DCGAN/dcgan.ipynb
class WaveGAN_Discriminator(nn.Module):
    """Discriminator containing 4 convolutional layers."""
    

    def __init__(self, image_size=128, conv_dim=d_conv_dim, n_phase = 2):
        
        if n_phase > 0:
            self.phaseshuffle = lambda x: apply_phaseshuffle(x,n_phase)
        else:
            self.phaseshuffle = lambda x: x
        
        super(WaveGAN_Discriminator, self).__init__()
        self.conv1 = conv1d(1, conv_dim, 25, bn=False)
        self.conv2 = conv1d(conv_dim, conv_dim*2, 25)
        self.conv3 = conv1d(conv_dim*2, conv_dim*4, 25)
        self.conv4 = conv1d(conv_dim*4, conv_dim*8, 25)
        self.conv5 = conv1d(conv_dim*8, conv_dim*16, 25)
        self.fc = nn.Linear(conv_dim*16*16,1)
        
#            conv(conv_dim*8, 1, int(image_size/16), 1, 0, False)
        
    def forward(self, x):                         # If image_size is 64, output shape is as below.
        print(x.size())
        out = F.leaky_relu(self.conv1(x), 0.2)    # (?, 64, 32, 32)
        print(out.size())
        out = self.phaseshuffle(out)
        print(out.size())
        out = F.leaky_relu(self.conv2(out), 0.2)  # (?, 128, 16, 16)
        out = self.phaseshuffle(out)
        print(out.size())
        out = F.leaky_relu(self.conv3(out), 0.2)  # (?, 256, 8, 8)
        out = self.phaseshuffle(out)
        print(out.size())
        out = F.leaky_relu(self.conv4(out), 0.2)  # (?, 512, 4, 4)
        out = self.phaseshuffle(out)
        print(out.size())
        out = F.leaky_relu(self.conv5(out), 0.2)  # (?, 512, 4, 4)
        out = self.phaseshuffle(out)
        print(out.size())
        out = out.view(out.size(0), 256 * d_conv_dim)
        out = self.phaseshuffle(out)
        print(out.size())
        out = F.sigmoid(self.fc(out))
        out = self.phaseshuffle(out)
        print(out.size())
        #phase_shuffle
#        out = phaseShuffle(out)
        out = out.squeeze()
        print(out.size())
        
        
        
        
        
#         out = self.fc(out).squeeze() # Least Square
        return out

In [82]:
wavegan_d = WaveGAN_Discriminator(n_phase = 2)

In [83]:
wavegan_d.forward(output)

torch.Size([5, 1, 16384])
torch.Size([5, 64, 4096])
-1 1 1 4096
phase
torch.Size([5, 64, 4096])
torch.Size([64, 4096])


NotImplementedError: Only 3D, 4D, 5D padding with non-constant padding are supported for now

In [17]:
criterion_vanillia = nn.BCELoss()

d_optimizer = torch.optim.Adam(D.parameters(), lr=lr, betas=(0.5,0.9))
g_optimizer = torch.optim.Adam(G.parameters(), lr=lr, betas=(0.5,0.9))


NameError: name 'D' is not defined

# Postprocessing Filter

# PhaseShuffle

In [39]:
a = [[1,2,3,4,5],[6,7,8,9,10]]
print(np.pad(a, [[0, 0], [1, 0], [0, 0]],'reflect'))
'''
reflect_type : {‘even’, ‘odd’}, optional

Used in ‘reflect’, and ‘symmetric’.
The ‘even’ style is the default with an unaltered reflection around the edge value. 
For the ‘odd’ style,
the extented part of the array is created by subtracting the reflected values  from two times the edge value.

'''

ValueError: Unable to create correctly shaped tuple from [[0, 0], [1, 0], [0, 0]]